In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
from tqdm.auto import tqdm

from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.ensemble import ExtraTreesClassifier, ExtraTreesRegressor, RandomForestClassifier, RandomForestRegressor,\
    AdaBoostClassifier, AdaBoostRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier, CatBoostRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from xgboost import XGBClassifier, XGBRegressor
import torch
from torch import nn
from torch.utils.data import DataLoader
import optuna

from library.Data_Preprocessing import Balance_Ratio, train_col
from library.Imbalance_Sampling import label_divide, resampling_dataset
from library.Aging_Score_Contour import score1
from library.AdaBoost import train_set, multiple_set, multiple_month, line_chart, cf_matrix, AUC, PR_curve, \
     multiple_curve, PR_matrix, best_threshold, all_optuna, optuna_history, AdaBoost_creator 
from library.XGBoost import XGBoost_creator
from library.LightGBM import LightGBM_creator
from library.CatBoost import CatBoost_creator
from library.RandomForest import RandomForest_creator
from library.ExtraTrees import ExtraTrees_creator
from library.NeuralNetwork import RunhistSet, NeuralNetworkC, trainingC
from library.StackingCV_Scheme3 import optimize_base, stratified_data, runall_LR, runall_RidgeR, stackingCV_creator, \
     vif, correlation_plot, rank_importance, LR

os.chdir('C:/Users/user/Desktop/Darui_R08621110')  
os.getcwd()

Using cpu device.


'C:\\Users\\user\\Desktop\\Darui_R08621110'

## Function Definition

### Load Hyperparameters

In [2]:
# load hyperparameters of all base learners in a dictionary
def load_hyper(num_set, date, model_list, iter_list, filename, mode, TPE_multi) :
    
    sampler = 'multivariate-TPE' if TPE_multi else 'univariate-TPE'
    allset_dict = {}
    for i, model in enumerate(model_list):
        allset_dict[model] = dict()
        with open(f'hyperparameter/{date}/{filename}_{model}{mode}_{sampler}_{iter_list[i]}.data', 'rb') as f:
            allset_dict[model] = pickle.load(f)
        
    return allset_dict

### Transform Data by Base Learners

In [3]:
# input training data to the base learners and output the outcome
def transform_train(train_data, mode, base_param, cv, add_origin = False):
    
    model_list = list(base_param.keys())
    set_list = list(base_param[model_list[0]].keys())
    set_dict = {}
    for i in tqdm(set_list):

        train_x_dict, train_y_dict, valid_x_dict, valid_y_dict = stratified_data(train_data[i], cv = cv)
        all_cv = pd.DataFrame()
        for j in tqdm(range(cv)):

            model_predict = valid_x_dict[j].copy().reset_index(drop = True) if add_origin else pd.DataFrame()
            if mode == 'C':          
                if 'NeuralNetwork' in model_list:
                    temp_train = RunhistSet(train_x_dict[j], train_y_dict[j])
                    temp_valid = RunhistSet(valid_x_dict[j], valid_y_dict[j])
                    train_loader = DataLoader(temp_train, 
                                              batch_size = base_param['NeuralNetwork'][i]['batch_size'], 
                                              shuffle = True)
                    valid_loader = DataLoader(temp_valid, batch_size = len(valid_x_dict[j]), shuffle = False)
                    nn_model = NeuralNetworkC(dim = train_x_dict[j].shape[1])
                    optimizer = torch.optim.Adam(nn_model.parameters(), 
                                                 lr = base_param['NeuralNetwork'][i]['learning_rate'], 
                                                 weight_decay = base_param['NeuralNetwork'][i]['weight_decay'])
                    criterion = nn.CrossEntropyLoss(
                        weight = torch.tensor([1-base_param['NeuralNetwork'][i]['bad_weight'], 
                                               base_param['NeuralNetwork'][i]['bad_weight']])).to('cpu')
                    network, _, _ = trainingC(nn_model, train_loader, train_loader, optimizer, criterion, epoch = 100, 
                                              early_stop = 10)
                    for x, y in valid_loader:
                        output = network(x)
                        predict_y = output.data[:, 1]
                    predict = pd.DataFrame({'N': predict_y.numpy()})
                    model_predict = pd.concat([model_predict, predict], axis = 1)
                
                if 'XGBoost' in model_list:                     
                    clf = XGBClassifier(**base_param['XGBoost'][i], use_label_encoder = False, n_jobs = -1)
                    clf.fit(train_x_dict[j], train_y_dict[j])
                    predict_y = clf.predict_proba(valid_x_dict[j])
                    predict = pd.DataFrame({'X': predict_y[:, 0]})
                    model_predict = pd.concat([model_predict, predict], axis = 1)

                if 'LightGBM' in model_list:                        
                    clf = LGBMClassifier(**base_param['LightGBM'][i])
                    clf.fit(train_x_dict[j], train_y_dict[j])
                    predict_y = clf.predict_proba(valid_x_dict[j])
                    predict = pd.DataFrame({'L': predict_y[:, 0]})
                    model_predict = pd.concat([model_predict, predict], axis = 1)

                if 'CatBoost' in model_list:
                    clf = CatBoostClassifier(**base_param['CatBoost'][i])
                    clf.fit(train_x_dict[j], train_y_dict[j])
                    predict_y = clf.predict_proba(valid_x_dict[j])
                    predict = pd.DataFrame({'C': predict_y[:, 0]})
                    model_predict = pd.concat([model_predict, predict], axis = 1)

                if 'AdaBoost' in model_list:
                    tree_param = {
                        'base_estimator': DecisionTreeClassifier(
                            max_depth = base_param['AdaBoost'][i]['max_depth']
                        )}
                    boost_param = dict(
                        (key, base_param['AdaBoost'][i][key]) for key in ['learning_rate', 'n_estimators']
                    )
                    boost_param.update(tree_param)
                    clf = AdaBoostClassifier(**boost_param)
                    clf.fit(train_x_dict[j], train_y_dict[j])
                    predict_y = clf.predict_proba(valid_x_dict[j])
                    predict = pd.DataFrame({'A': predict_y[:, 0]})
                    model_predict = pd.concat([model_predict, predict], axis = 1)

                if 'RandomForest' in model_list:
                    clf = RandomForestClassifier(**base_param['RandomForest'][i])
                    clf.fit(train_x_dict[j], train_y_dict[j])
                    predict_y = clf.predict_proba(valid_x_dict[j])
                    predict = pd.DataFrame({'R': predict_y[:, 0]})
                    model_predict = pd.concat([model_predict, predict], axis = 1)

                if 'ExtraTrees' in model_list:
                    clf = ExtraTreesClassifier(**base_param['ExtraTrees'][i])
                    clf.fit(train_x_dict[j], train_y_dict[j])
                    predict_y = clf.predict_proba(valid_x_dict[j])
                    predict = pd.DataFrame({'E': predict_y[:, 0]})
                    model_predict = pd.concat([model_predict, predict], axis = 1)

            elif mode == 'R':

                if 'XGBoost' in model_list:
                    reg = XGBRegressor(**base_param[i]['XGBoost'], use_label_encoder = False, n_jobs = -1)
                    reg.fit(train_x_dict[j], train_y_dict[j])
                    predict_y = reg.predict(valid_x_dict[j])
                    predict = pd.DataFrame({'X': predict_y})
                    model_predict = pd.concat([model_predict, predict], axis = 1)

                if 'LightGBM' in model_list:
                    reg = LGBMRegressor(**base_param[i]['LightGBM'])
                    reg.fit(train_x_dict[j], train_y_dict[j])
                    predict_y = reg.predict(valid_x_dict[j])
                    predict = pd.DataFrame({'L': predict_y})
                    model_predict = pd.concat([model_predict, predict], axis = 1)

                if 'CatBoost' in model_list:
                    reg = CatBoostRegressor(**base_param[i]['CatBoost'])
                    reg.fit(train_x_dict[j], train_y_dict[j])
                    predict_y = reg.predict(valid_x_dict[j])
                    predict = pd.DataFrame({'C': predict_y})
                    model_predict = pd.concat([model_predict, predict], axis = 1)

                if 'AdaBoost' in model_list:
                    tree_param = {
                        'base_estimator': DecisionTreeRegressor(
                            max_depth = base_param[i]['AdaBoost']['max_depth']
                        )}
                    boost_param = dict(
                        (key, base_param[i]['AdaBoost'][key]) for key in ['learning_rate', 'n_estimators']
                    )
                    boost_param.update(tree_param)
                    reg = AdaBoostRegressor(**boost_param)
                    reg.fit(train_x_dict[j], train_y_dict[j])
                    predict_y = reg.predict(valid_x_dict[j])
                    predict = pd.DataFrame({'A': predict_y})
                    model_predict = pd.concat([model_predict, predict], axis = 1)

                if 'RandomForest' in model_list:
                    reg = RandomForestRegressor(**base_param[i]['RandomForest'])
                    reg.fit(train_x_dict[j], train_y_dict[j])
                    predict_y = reg.predict(valid_x_dict[j])
                    predict = pd.DataFrame({'R': predict_y})
                    model_predict = pd.concat([model_predict, predict], axis = 1)

                if 'ExtraTrees' in model_list:
                    reg = ExtraTreesRegressor(**base_param[i]['ExtraTrees'])
                    reg.fit(train_x_dict[j], train_y_dict[j])
                    predict_y = reg.predict(valid_x_dict[j])
                    predict = pd.DataFrame({'E': predict_y})
                    model_predict = pd.concat([model_predict, predict], axis = 1)

            test_label = valid_y_dict[j].reset_index(drop = True)
            done_cv = pd.concat([model_predict, test_label], axis = 1)
            all_cv = pd.concat([all_cv, done_cv], axis = 0)

        set_dict[i] = all_cv
    
    return set_dict


# input testing data to the base learners and output the outcome
def transform_test(train_data, test_data, mode, base_param, add_origin = False):
    
    model_list = list(base_param.keys())
    set_list = list(base_param[model_list[0]].keys())
    test_dict = {}
    for i in tqdm(set_list):
        
        train_x, train_y, test_x, test_y = label_divide(train_data[i], test_data, train_only = False)
        model_predict = test_x.copy().reset_index(drop = True) if add_origin else pd.DataFrame()
        if mode == 'C':

            if 'NeuralNetwork' in model_list:
                temp_train = RunhistSet(train_x, train_y)
                temp_test = RunhistSet(test_x, test_y)
                train_loader = DataLoader(temp_train, 
                                          batch_size = base_param['NeuralNetwork'][i]['batch_size'], 
                                          shuffle = True)
                test_loader = DataLoader(temp_test, batch_size = len(test_x), shuffle = False)
                nn_model = NeuralNetworkC(dim = train_x.shape[1])
                optimizer = torch.optim.Adam(nn_model.parameters(), 
                                             lr = base_param['NeuralNetwork'][i]['learning_rate'], 
                                             weight_decay = base_param['NeuralNetwork'][i]['weight_decay'])
                criterion = nn.CrossEntropyLoss(
                    weight = torch.tensor([1-base_param['NeuralNetwork'][i]['bad_weight'], 
                                           base_param['NeuralNetwork'][i]['bad_weight']])).to('cpu')
                network, _, _ = trainingC(nn_model, train_loader, train_loader, optimizer, criterion, epoch = 100, 
                                          early_stop = 10)
                for X, Y in test_loader:
                    X, Y = X.float(), Y.long()
                    output = network(X)
                    predict_y = output.data[:, 1]
                predict = pd.DataFrame({'N': predict_y.numpy()})
                model_predict = pd.concat([model_predict, predict], axis = 1)
            
            if 'XGBoost' in model_list:
                clf = XGBClassifier(**base_param['XGBoost'][i], use_label_encoder = False, n_jobs = -1)
                clf.fit(train_x, train_y)
                predict_y = clf.predict_proba(test_x)
                predict = pd.DataFrame({'X': predict_y[:, 0]})
                model_predict = pd.concat([model_predict, predict], axis = 1)

            if 'LightGBM' in model_list:
                clf = LGBMClassifier(**base_param['LightGBM'][i])
                clf.fit(train_x, train_y)
                predict_y = clf.predict_proba(test_x)
                predict = pd.DataFrame({'L': predict_y[:, 0]})
                model_predict = pd.concat([model_predict, predict], axis = 1)

            if 'CatBoost' in model_list:
                clf = CatBoostClassifier(**base_param['CatBoost'][i])
                clf.fit(train_x, train_y)
                predict_y = clf.predict_proba(test_x)
                predict = pd.DataFrame({'C': predict_y[:, 0]})
                model_predict = pd.concat([model_predict, predict], axis = 1)

            if 'AdaBoost' in model_list:
                tree_param = {
                    'base_estimator': DecisionTreeClassifier(
                        max_depth = base_param['AdaBoost'][i]['max_depth']
                    )}
                boost_param = dict(
                    (key, base_param['AdaBoost'][i][key]) for key in ['learning_rate', 'n_estimators']
                )
                boost_param.update(tree_param)
                clf = AdaBoostClassifier(**boost_param)
                clf.fit(train_x, train_y)
                predict_y = clf.predict_proba(test_x)
                predict = pd.DataFrame({'A': predict_y[:, 0]})
                model_predict = pd.concat([model_predict, predict], axis = 1)

            if 'RandomForest' in model_list:
                clf = RandomForestClassifier(**base_param['RandomForest'][i])
                clf.fit(train_x, train_y)
                predict_y = clf.predict_proba(test_x)
                predict = pd.DataFrame({'R': predict_y[:, 0]})
                model_predict = pd.concat([model_predict, predict], axis = 1)

            if 'ExtraTrees' in model_list:
                clf = ExtraTreesClassifier(**base_param['ExtraTrees'][i])
                clf.fit(train_x, train_y)
                predict_y = clf.predict_proba(test_x)
                predict = pd.DataFrame({'E': predict_y[:, 0]})
                model_predict = pd.concat([model_predict, predict], axis = 1)

        elif mode == 'R':

            if 'XGBoost' in model_list:
                reg = XGBRegressor(**base_param[i]['XGBoost'], use_label_encoder = False, n_jobs = -1)
                reg.fit(train_x, train_y)
                predict_y = reg.predict(test_x)
                predict = pd.DataFrame({'X': predict_y})
                model_predict = pd.concat([model_predict, predict], axis = 1)

            if 'LightGBM' in model_list:
                reg = LGBMRegressor(**base_param[i]['LightGBM'])
                reg.fit(train_x, train_y)
                predict_y = reg.predict(test_x)
                predict = pd.DataFrame({'L': predict_y})
                model_predict = pd.concat([model_predict, predict], axis = 1)

            if 'CatBoost' in model_list:
                reg = CatBoostRegressor(**base_param[i]['CatBoost'])
                reg.fit(train_x, train_y)
                predict_y = reg.predict(test_x)
                predict = pd.DataFrame({'C': predict_y})
                model_predict = pd.concat([model_predict, predict], axis = 1)

            if 'AdaBoost' in model_list:
                tree_param = {
                    'base_estimator': DecisionTreeRegressor(
                        max_depth = base_param[i]['AdaBoost']['max_depth']
                    )}
                boost_param = dict(
                    (key, base_param[i]['AdaBoost'][key]) for key in ['learning_rate', 'n_estimators']
                )
                boost_param.update(tree_param)
                reg = AdaBoostRegressor(**boost_param)
                reg.fit(train_x, train_y)
                predict_y = reg.predict(test_x)
                predict = pd.DataFrame({'A': predict_y})
                model_predict = pd.concat([model_predict, predict], axis = 1)

            if 'RandomForest' in model_list:
                reg = RandomForestRegressor(**base_param[i]['RandomForest'])
                reg.fit(train_x, train_y)
                predict_y = reg.predict(test_x)
                predict = pd.DataFrame({'R': predict_y})
                model_predict = pd.concat([model_predict, predict], axis = 1)

            if 'ExtraTrees' in model_list:
                reg = ExtraTreesRegressor(**base_param[i]['ExtraTrees'])
                reg.fit(train_x, train_y)
                predict_y = reg.predict(test_x)
                predict = pd.DataFrame({'E': predict_y})
                model_predict = pd.concat([model_predict, predict], axis = 1)

        model_done = pd.concat([model_predict, test_y], axis = 1)
        test_dict[i] = model_done
        
    return test_dict

### Optuna

In [4]:
def stackingcv1_creator(train_data, mode, learner = 'LogisticRegression', num_valid = 5):
    return stackingCV_creator(train_data = train_data, mode = mode, learner = learner, num_valid = num_valid)

### Full Experiment

In [5]:
def full_stackingcv1(train_month, times):
    prob_dict = dict()
    result_df = pd.DataFrame()

    # load relabel datasets
    runhist = {}
    kinds = {}
    for i in train_month:
        runhist[f'm{i}'] = pd.read_csv(f'relabel_runhist_m{i}.csv', index_col = 'id').iloc[:, 1:]
        kinds[f'm{i}'] = pd.read_csv(f'kind_m{i}.csv').iloc[:, 2:-3]

    # do several times to average the random effect of resampling
    for i in tqdm(range(times)):
        # generate resampled datasets
        resampling_dataset(runhist = runhist, kinds = kinds, train_month = train_month, final_br = 1, num_os = 10)

        # load & prepare the resampled datasets 
        all_train = multiple_set(num_set = 10)
        all_train_x, all_train_y = train_set(all_train)
        all_test = pd.read_csv('test_runhist.csv').iloc[:, 2:]
        all_test_x, all_test_y = label_divide(all_test, None, 'GB', train_only = True)

        # optimization for each month of data
        base_param = optimize_base(train_data = {'all': all_train}, 
                                   mode = 'C', 
                                   TPE_multi = False, 
                                   base_list = ['NeuralNetwork', 'LightGBM', 'XGBoost'],
                                   iter_dict = {'LightGBM': 25, 'NeuralNetwork': 10, 'XGBoost': 25, 'CatBoost': 25, 
                                               'RandomForest': 20, 'ExtraTrees': 20},
                                   filename = f'runhist_array_m2m4_m5_3criteria_scheme1-{i}')
        
        # data transformation
        trans_train = transform_train(all_train, mode = 'C', base_param = base_param['all'], cv = 5, add_origin = False)
        trans_test = transform_test(all_train, all_test, mode = 'C', base_param = base_param['all'], add_origin = False)
        trans_train_x, trans_train_y = train_set(trans_train)
        trans_test_x, trans_test_y = train_set(trans_test) 
        trans_train['set0'] = {}
        
        # searching for hyperparameters
        best_param, _ = all_optuna(all_data = trans_train, 
                                   mode = 'C', 
                                   TPE_multi = False, 
                                   n_iter = 10,
                                   filename = f'runhist_array_m2m4_m5_3criteria_StackingCV1-{i}',
                                   creator = stackingcv1_creator)
        
        # store the probability predicted by the classifier 
        for j in best_param.keys():
            if i == 0:
                prob_dict[j] = pd.DataFrame()
            table, _ = LR(trans_train_x[j], trans_test_x[j], trans_train_y[j], trans_test_y[j], best_param[j], 
                          return_prob = True)
            prob_dict[j] = pd.concat([prob_dict[j], table[['predict']]], axis = 1)
            
    # average to get final prediction
    for j in best_param.keys():
        prediction = (prob_dict[j].apply(np.sum, axis = 1) >= 0.5).astype(int)
        result = pd.DataFrame(dict(truth = all_test_y, predict = prediction))
        table = cf_matrix(result, all_train_y[j])
        result_df = pd.concat([result_df, table]).rename(index = {0: f'data{j}'})
        
    return result_df

## Prediction

### Full Experiment

In [ ]:
training_month = range(2, 5)
table_setC = full_stackingcv1(training_month, times = 3)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Month 2:
# bad: 69
Size before Border: 39009
Distance = 1 ...
Distance = 2 ...
Distance = 3 ...
Distance = 4 ...
Distance = 5 ...
Distance = 6 ...
Distance = 7 ...
Distance = 8 ...
# over: 10
Distance = 9 ...
# over: 10
Distance = 10 ...
# over: 10
Distance = 11 ...
# over: 50
Distance = 12 ...
# over: 30
Distance = 13 ...
# over: 70
Distance = 14 ...
# over: 70
Distance = 15 ...
# over: 240
Distance = 16 ...
# over: 150
# over: 150
Size after Border: 39624

 Generating Dataset 2
Size after Undersampling: 1368
Size after Undersampling: (1368, 88) , Balance Ratio: 1.0
Size before Undersampling: 39009

 Generating Dataset 6
Size after Undersampling: 759
Size after Undersampling: 759
Distance = 1 ...
Distance = 2 ...
Distance = 3 ...
Distance = 4 ...
Distance = 5 ...
Distance = 6 ...
Distance = 7 ...
Distance = 8 ...
# over: 10
Distance = 9 ...
# over: 10
Distance = 10 ...
# over: 10
Distance = 11 ...
# over: 50
Distance = 12 ...
# over: 30
Distance = 13 ...
# over: 70
Distance = 14 ...
#

  0%|          | 0/1 [00:00<?, ?it/s]


Starting for CatBoost:



...

  0%|          | 0/150 [00:00<?, ?it/s]

Epoch 1: Train Loss = 0.5638780082747645
Epoch 2: Train Loss = 0.4799644634167424
Epoch 3: Train Loss = 0.44633764134701726
Epoch 4: Train Loss = 0.4241545933861873
Epoch 5: Train Loss = 0.41165101883778693
Recall = 0.9642857142857143, Aging Rate = 0.5948352626892253, Precision = 0.8184880239520959
Validation: Test Loss = 0.3869753769223221
Recall = 0.9700176366843033, Aging Rate = 0.5547640249332146, precision = 0.8828250401284109
Model in epoch 5 is saved.

Epoch 6: Train Loss = 0.3892845412979992
Epoch 7: Train Loss = 0.37882327603529525
Epoch 8: Train Loss = 0.37609679629414183
Epoch 9: Train Loss = 0.3636430881167774
Epoch 10: Train Loss = 0.36446490108913754
Recall = 0.9841269841269841, Aging Rate = 0.5445235975066786, Precision = 0.9125102207686018
Validation: Test Loss = 0.37504753650880135
Recall = 0.9581128747795414, Aging Rate = 0.5155832591273375, precision = 0.9382556131260794
Model in epoch 10 is saved.

Epoch 11: Train Loss = 0.3605929729615803
Epoch 12: Train Loss = 0.3

Epoch 93: Train Loss = 0.346200076549688
Epoch 94: Train Loss = 0.34491460894434345
Epoch 95: Train Loss = 0.3476584913575639
Recall = 0.9933862433862434, Aging Rate = 0.5253784505788067, Precision = 0.9546610169491525
Validation: Test Loss = 0.3402209941128589
Recall = 0.9973544973544973, Aging Rate = 0.5173642030276047, precision = 0.9733218588640276

Epoch 96: Train Loss = 0.3445361222099855
Epoch 97: Train Loss = 0.34727753931034494
Epoch 98: Train Loss = 0.3437383737770023
Epoch 99: Train Loss = 0.34459435815059597
Epoch 100: Train Loss = 0.3475092710762924
Recall = 0.9916225749559083, Aging Rate = 0.523820124666073, Precision = 0.9558011049723757
Validation: Test Loss = 0.3417300856070973
Recall = 0.9964726631393298, Aging Rate = 0.5229296527159395, precision = 0.9621115368241805

Epoch 101: Train Loss = 0.34696159207279514
Epoch 102: Train Loss = 0.3484883372367543
Epoch 103: Train Loss = 0.3458051652591991
Epoch 104: Train Loss = 0.3477672893663442
Epoch 105: Train Loss = 0.344

  0%|          | 0/150 [00:00<?, ?it/s]

Epoch 1: Train Loss = 0.6603423383017257
Epoch 2: Train Loss = 0.5957662996073865
Epoch 3: Train Loss = 0.5541844836909752
Epoch 4: Train Loss = 0.53875514830422
Epoch 5: Train Loss = 0.5315851984232107
Recall = 0.8897389738973898, Aging Rate = 0.6077471059661621, Precision = 0.7241758241758242
Validation: Test Loss = 0.5243144019947239
Recall = 0.9014401440144014, Aging Rate = 0.6101959038290294, precision = 0.7307551988325429
Model in epoch 5 is saved.

Epoch 6: Train Loss = 0.5258701373876893
Epoch 7: Train Loss = 0.5226336032921782
Epoch 8: Train Loss = 0.5190778477639889
Epoch 9: Train Loss = 0.5163509452438524
Epoch 10: Train Loss = 0.5140318157199442
Recall = 0.9081908190819082, Aging Rate = 0.6095280498664292, Precision = 0.7370343316289262
Validation: Test Loss = 0.5093112504599037
Recall = 0.9117911791179117, Aging Rate = 0.6073018699910953, precision = 0.7426686217008798
Model in epoch 10 is saved.

Epoch 11: Train Loss = 0.5110344598044908
Epoch 12: Train Loss = 0.508643120

In [ ]:
line_chart(table_setC, title = 'StackingCV Scheme 1 Classifier')
table_setC

### Export

In [ ]:
savedate = '20220601'
TPE_multi = False

table_setC['sampler'] = 'multivariate-TPE' if TPE_multi else 'univariate-TPE'
table_setC['model'] = 'StackingCV1'
with pd.ExcelWriter(f'{savedate}_Classifier.xlsx', mode = 'a') as writer:
    table_setC.to_excel(writer, sheet_name = 'StackingCV1')